In [7]:
# Linear Regression With Stochastic Gradient Descent for Wine Quality
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [13]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
  


# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


In [10]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate root mean squared error
def rmse_metric(actual, predicted):
	sum_error = 0.0
	for i in range(len(actual)):
		prediction_error = predicted[i] - actual[i]
		sum_error += (prediction_error ** 2)
	mean_error = sum_error / float(len(actual))
	return sqrt(mean_error)


In [15]:
lasso_lambda = 2
ridge_lambda = 2
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		rmse = rmse_metric(actual, predicted)
		scores.append(rmse)
	return scores

# Make a prediction with coefficients
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * row[i]
	return yhat

# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			yhat = predict(row, coef) #Sigma2(xijBij)
			error = yhat - row[-1] 
      #error = yhat - row[-1] + lasso_lambda   #for lasso _regression
      #error = yhat - row[-1] + 2*ridge_lambda  #for ridge regression
			coef[0] = coef[0] - l_rate * error
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] - l_rate * error * row[i] # weight update
			# print(l_rate, n_epoch, error)
	return coef

# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
	predictions = list()
	coef = coefficients_sgd(train, l_rate, n_epoch)
	for row in test:
		yhat = predict(row, coef)
		predictions.append(yhat)
	return(predictions)


In [18]:

# Linear Regression on wine quality dataset
seed(1)
# load and prepare data
filename = 'winequality-white.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

[9.4;0.29;0.55;2.2;0.05;17;119;0.9962;3.12;0.69;10.3;4] => 0
[7.4;0.28;0.25;11.9;0.053;25;148;0.9976;3.1;0.62;9.2;5] => 1
[7.7;0.43;1;19.95;0.032;42;164;0.99742;3.29;0.5;12;6] => 2
[9.2;0.28;0.49;11.8;0.042;29;137;0.998;3.1;0.34;10.1;4] => 3
[5.4;0.205;0.16;12.55;0.051;31;115;0.99564;3.4;0.38;10.8;6] => 4
[6.1;0.15;0.29;6.2;0.046;39;151;0.99471;3.6;0.44;10.6;6] => 5
[6.8;0.36;0.24;4.6;0.039;24;124;0.9909;3.27;0.34;12.6;7] => 6
[7.1;0.12;0.3;3.1;0.018;15;37;0.99004;3.02;0.52;11.9;7] => 7
[5.9;0.3;0.29;1.1;0.036;23;56;0.9904;3.19;0.38;11.3;5] => 8
[6.4;0.17;0.32;2.4;0.048;41;200;0.9938;3.5;0.5;9.7;6] => 9
[6.2;0.235;0.34;1.9;0.036;4;117;0.99032;3.4;0.44;12.2;5] => 10
[6.4;0.26;0.25;10.7;0.046;66;179;0.99606;3.17;0.55;9.9;6] => 11
[5.9;0.26;0.27;18.2;0.048;52;168;0.9993;3.35;0.44;9.4;5] => 12
[7.9;0.46;0.32;4.1;0.033;40;138;0.9912;3.18;0.44;12.8;7] => 13
[6.1;0.38;0.42;5;0.016;31;113;0.99007;3.15;0.31;12.4;7] => 14
[6.45;0.14;0.42;1.2;0.05;51;129;0.99116;3.27;0.69;11.1;7] => 15
[5.5;0.24;